In [ ]:
%load_ext autoreload
%autoreload 2

In [20]:
### IMPORTS ###

import b3d
import jax.numpy as jnp
import os
from b3d import Mesh, Pose
import jax
import genjax
from genjax import Pytree
import rerun as rr
from b3d.modeling_utils import uniform_discrete, uniform_pose, gaussian_vmf
import matplotlib.pyplot as plt
from functools import partial
import importlib
from ipywidgets import interact
import ipywidgets as widgets
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
from genjax import SelectionBuilder as S
from genjax import ChoiceMapBuilder as C

b3d.rr_init("dynamics")

In [ ]:
### Loading data ###

scene_id = 55
FRAME_RATE = 50
ycb_dir = os.path.join(b3d.get_assets_path(), "bop/ycbv")
print(f"Scene {scene_id}")
b3d.reload(b3d.io.data_loader)
num_scenes = b3d.io.data_loader.get_ycbv_num_test_images(ycb_dir, scene_id)

# image_ids = [image] if image is not None else range(1, num_scenes, FRAME_RATE)
image_ids = range(1, num_scenes + 1, FRAME_RATE)
all_data = b3d.io.data_loader.get_ycbv_test_images(ycb_dir, scene_id, image_ids)

meshes = [
    Mesh.from_obj_file(
        os.path.join(ycb_dir, f'models/obj_{f"{id + 1}".rjust(6, "0")}.ply')
    ).scale(0.001)
    for id in all_data[0]["object_types"]
]

image_height, image_width = all_data[0]["rgbd"].shape[:2]
fx,fy,cx,cy = all_data[0]["camera_intrinsics"]
scaling_factor = 1.0
renderer = b3d.renderer.renderer_original.RendererOriginal(
    image_width * scaling_factor, image_height * scaling_factor, fx * scaling_factor, fy * scaling_factor, cx * scaling_factor, cy * scaling_factor, 0.01, 2.0
)
b3d.viz_rgb(all_data[0]["rgbd"])

In [61]:
from b3d.chisight.dynamic_object_model.dynamic_object_model import dynamic_object_generative_model, viz_trace, info_from_trace
from b3d.chisight.dynamic_object_model.dynamic_object_inference import inference_step
b3d.reload(b3d.chisight.dynamic_object_model.dynamic_object_model)
b3d.reload(b3d.chisight.dynamic_object_model.dynamic_object_inference)

In [ ]:
### Set up model ###

T = 0
b3d.rr_set_time(T)
O1JECT_INDEX = 1

template_pose = all_data[T]["camera_pose"].inv() @ all_data[T]["object_poses"][OBJECT_INDEX]
rendered_rgbd = renderer.render_rgbd_from_mesh(meshes[OBJECT_INDEX].transform(template_pose))
xyz_rendered = b3d.xyz_from_depth(rendered_rgbd[..., 3], fx, fy, cx, cy)

fx, fy, cx, cy = all_data[T]["camera_intrinsics"]
xyz_observed = b3d.xyz_from_depth(all_data[T]["rgbd"][..., 3], fx, fy, cx, cy)
mask = all_data[T]["masks"][OBJECT_INDEX] * (xyz_observed[..., 2] > 0) * (jnp.linalg.norm(xyz_rendered - xyz_observed, axis=-1) < 0.01)
model_vertices = template_pose.inv().apply(xyz_rendered[mask])
model_colors = vertex_attributes=all_data[T]["rgbd"][..., :3][mask]

hyperparams = {
    "vertices": model_vertices,
    "max_color_shift": Pytree.const(0.05),
    "max_pose_position_shift": Pytree.const(0.1),
    "fx": fx,
    "fy": fy,
    "cx": cx,
    "cy": cy,
    "image_height": Pytree.const(image_height),
    "image_width": Pytree.const(image_width)
}

previous_state = {
    "pose": template_pose,
    "colors": model_colors,
}

choicemap = genjax.ChoiceMap.d(
    {
        "pose": template_pose,
        "colors": model_colors,

        "rgbd": all_data[T]["rgbd"],

        "color_variance": 0.05,
        "depth_variance": 0.005,
        "color_outlier_probability": jnp.ones(len(model_vertices)) * 0.01,
        "depth_outlier_probability": jnp.ones(len(model_vertices)) * 0.01,
    }
)
key = jax.random.PRNGKey(0)
trace, _ = dynamic_object_generative_model.importance(key, choicemap, (hyperparams, previous_state))
viz_trace(trace, 0)
results = {}
info = info_from_trace(trace)
trace.get_score()

In [65]:

### Run inference ###
for T in tqdm(range(len(all_data))):
    trace = inference_step(trace, key, all_data[T]["rgbd"])
    viz_trace(trace, T)
    results[T] = trace

viz_trace(trace, T, ground_truth_vertices=meshes[OBJECT_INDEX].vertices, ground_truth_pose=all_data[T]["camera_pose"].inv() @ all_data[T]["object_poses"][OBJECT_INDEX])

  0%|          | 0/37 [00:00<?, ?it/s]

100%|██████████| 37/37 [00:36<00:00,  1.00it/s]


In [ ]:
info_from_trace(trace)["scores"] < 0.0

In [304]:
info_from_trace(trace)["scores"] < 0.0

Array([ 1.8246567 ,  0.98210424, -0.11121082, ..., -8.72367   ,
        2.7278347 ,  2.7573478 ], dtype=float32)